In [145]:
import requests
from datetime import datetime as dt
import pandas as pd
from dateutil import relativedelta
import sys
import os
import pygsheets
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pygsheets
import time
from bs4 import BeautifulSoup
import glob
import json

In [146]:
client = pygsheets.authorize(service_file = "C:/Users/Tường/Desktop/refined-grammar-372209-b3fbc4b10ebf.json")
sh = client.open('Recruitment')
wks = sh.worksheet_by_title('CB Data')
recruitment_data = wks.get_as_df()
print('Get old data')

Get old data


In [147]:
options = Options()
options.add_experimental_option('prefs', {
"download.default_directory": r"C:\Users\Tường\Desktop\glints\cv", #Change default directory for downloads
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome(options = options)
driver.get('https://careerbuilder.vn/vi/employers/hrcentral/manageresume')

In [148]:
f = open('data.json')

data = json.load(f)

username = data['username']
password = data['password']

user_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[1]/div[2]/input')
user_field.send_keys(username)
time.sleep(5)

pass_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[2]/div[2]/input')
pass_field.send_keys(password)
time.sleep(5)

login_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[3]/div/button')
login_field.click()
time.sleep(5)
print('Login')

Login


In [149]:
job_link = []

for i in range(2,100):
    try:
        job_input = driver.find_element(By.XPATH,'//*[@id="select-folder"]')
        job_input.click()
        job =  driver.find_element(By.XPATH, f'//*[@id="select-folder"]/option[{i}]')
        job.click()
        url = driver.current_url
        index = str(url).find('/*/0/0/*/*/7/2/6/2/0/desc/hr.1661228713/1')
        id = str(url)[index-8:index]
        link = r'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/' + id +r'/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1'
        job_link.append(link)
    except:
        job_input = driver.find_element(By.XPATH,'//*[@id="select-folder"]')
        job_input.click()
        job =  driver.find_element(By.XPATH, f'//*[@id="select-folder"]/option[1]')
        job.click()
        url = driver.current_url
        index = str(url).find('/*/0/0/*/*/7/2/6/2/0/desc/hr.1661228713/1')
        id = str(url)[index-8:index]
        link = r'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/' + id +r'/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1'
        job_link.append(link)
        break



In [150]:
job_link

['https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/35C042C7/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1',
 'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/35C03E55/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1',
 'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/35C03ED5/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1',
 'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/35C042B0/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1']

In [151]:
candidate_url = []

for id in job_link:

    driver.get(link)

    page_navi = driver.find_element(By.XPATH, f'//*[@id="tab-1"]/div/div[2]/div[2]/div/p').text
    if page_navi != "":
        no_candidate = int(page_navi.split(' ')[6])

 
        if no_candidate % 20 == 0:
            no_page = no_candidate//20
        else:
            no_page = (no_candidate//20) + 1

    
        page_lst = [i for i in range(1,no_page +1)]
        if len(page_lst) >1:
            for page in page_lst:
                page = driver.find_element(By.XPATH,f'//*[@id="tab-1"]/div/div[3]/div[2]/div/ul/li[{page}]/a')
                page.click()
                pagesource = BeautifulSoup(driver.page_source)
                candidate = pagesource.find_all('a',class_ = 'name')
                for i in candidate:
                    up_index = str(i).find('href="') + 6
                    low_index = str(i).find('" target=')
                    candidate_url.append(str(i)[up_index:low_index])
        else:
            pagesource = BeautifulSoup(driver.page_source)
            candidate = pagesource.find_all('a',class_ = 'name')
            for i in candidate:
                up_index = str(i).find('href="') + 6
                low_index = str(i).find('" target=')
                candidate_url.append(str(i)[up_index:low_index])
print('Get candidate url:', len(candidate_url))

Get candidate url: 4


In [152]:
candidate_url

['https://careerbuilder.vn/vi/employers/hrcentral/manage_resumes/resumes_detail/1/35C042B0/35C042B0/374C05A6/374C05A6/36BB3813/35E13FE3/hr.1661228713/36BB3813/35E13FE3',
 'https://careerbuilder.vn/vi/employers/hrcentral/manage_resumes/resumes_detail/1/35C042B0/35C042B0/374C05A6/374C05A6/36BB3813/35E13FE3/hr.1661228713/36BB3813/35E13FE3',
 'https://careerbuilder.vn/vi/employers/hrcentral/manage_resumes/resumes_detail/1/35C042B0/35C042B0/374C05A6/374C05A6/36BB3813/35E13FE3/hr.1661228713/36BB3813/35E13FE3',
 'https://careerbuilder.vn/vi/employers/hrcentral/manage_resumes/resumes_detail/1/35C042B0/35C042B0/374C05A6/374C05A6/36BB3813/35E13FE3/hr.1661228713/36BB3813/35E13FE3']

In [153]:
old_values = recruitment_data['id'].values
new_candidate = []
for i in candidate_url:
    if i[-17:-9] not in old_values:
        new_candidate.append(i)
print('New candidate start from 2023-02-03:', len(new_candidate))

New candidate: 4


In [154]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

In [155]:
position = []
name = []
mail = []
phone_number = []
company = []
id = []
year = []
month = []
date = []

for candidate in new_candidate:
    driver.get(candidate)

    candidate_position = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[1]/nav/ol/li[2]').text
    position.append(candidate_position)
    if candidate_position.lower() == 'accountant' or candidate_position.lower() == 'customer service associate':
        company.append('Genkin')
    else:
        company.append('Maido')

    candidate_name = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[1]/div[1]/ul/li[1]/p[2]').text
    name.append(candidate_name)

    apply_date = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[3]/ul/li[3]/p[2]').text
    date.append(apply_date)
    month.append(apply_date.split('-')[1])
    year.append(apply_date.split('-')[2])

    if driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[1]/p[1]/strong').text.strip() == "ĐTDĐ :":
        phone_number.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[1]/p[2]').text)
        mail.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[2]/p[2]').text)
    else:
        phone_number.append("")
        mail.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li/p[2]').text)

    candidate_id = candidate[-17:-9]
    id.append(candidate_id)
print('Get data')

Get data


In [156]:
df =pd.DataFrame({'date':date,
                  'month':month,
                  'year':year,
                  'company': company,
                  'name':name,
                  'mail':mail,
                  'phone_number':phone_number,
                  'position':position,
                  'id':id
                 }, columns = ['id','date','month','year','company','name','mail','phone_number','position'])

In [157]:

output_dir = 'C:/Users/Tường/Desktop/glints/cv'

for i in range(0,len(df)):

    candidate_name = df.iloc[i]['name']
    candidate_id = df.iloc[i]['id']
    candidate_position = df.iloc[i]['position']
    apply_date = df.iloc[i]['date']
    
    download_url = f'https://careerbuilder.vn/vi/employers/popup/downloadresume?resume_id={candidate_id}'
    driver.get(download_url)

print('CV downloaded')

CV downloaded


In [158]:
new_name = ['_'.join(position.split(' ')) +"("+date+')' for position,date in zip(df['position'],df['date'])]
id = df['id'].values

rename = {}

for i in range(0,len(df)):
    rename[id[i]] = new_name[i]


In [160]:
upload_file_list = []
file_name_dict = {}
list_of_files = glob.glob('C:/Users/Tường/Desktop/glints/cv/*.pdf')
for i in list_of_files:
    index = str(i)[-12:-4]
    if index in rename.keys():
        new_name = str(i).replace(index,rename[index]).replace('.pdf','')
        if(not os.path.exists(new_name)):    
            os.rename(i,new_name)
        file_name = os.path.basename(new_name)
        file_name_dict[index] = file_name
        upload_file_list.append(new_name)
print(f'{len(upload_file_list)} downloaded files have renamed')

1 downloaded files have renamed


In [161]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()    
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
print('Authorization done')
targetDirID = '1-ORa9e9s1XY_NE8SOwXQLJ0fXekysd9F'
 
exist_file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(targetDirID)}).GetList()

for upload_file in upload_file_list:
	if(not os.path.exists(upload_file)):
		continue
		
	fileName = os.path.basename(upload_file)
	for file1 in exist_file_list:
		if file1['title'] == fileName:
			file1.Delete()
		
	gfile = drive.CreateFile({'parents': [{'id': targetDirID}], 'title': fileName})
	# Read file and set it as the content of this instance.
	gfile.SetContentFile(upload_file)
	gfile.Upload() # Upload the file.
print(f'{len(upload_file_list)} files uploaded')

Authorization done
1 files uploaded


In [162]:
name_lst = []
cv_link = []
files = drive.ListFile({"q": "'" + targetDirID + "' in parents and mimeType!='application/vnd.google-apps.folder'"}).GetList()
for file in files:
    keys = file.keys()
    if file['shared'] and 'alternateLink' in keys:
        link = file['alternateLink']
    else:
        link = 'No Link Available. Check your sharing settings.'
    name = file['title']
    name_lst.append(name)
    cv_link.append(link)
    
cv_df = pd.DataFrame({'file_name':name_lst,'cv':cv_link}, columns=['file_name','cv'])
cv_df.drop_duplicates(subset='file_name', keep="last",inplace = True)

In [163]:
cv_df

,file_name,cv
0,Nguyen_Hoang_Ngoc_CUSTOMER_SERVICE_ASSOCIATE(0...,https://drive.google.com/file/d/16btuq69P-tAFj...
1,Tinh_Tinh_Junior_Graphic_Designer(03-02-2023),https://drive.google.com/file/d/1-I4rk91Di6OR5...
2,nguyen_hoang_Junior_Graphic_Designer(03-02-2023),https://drive.google.com/file/d/1JnJuOVp8GDFSf...
3,Ly_Huynh_To_Nhu_Junior_Account_Executive(03-02...,https://drive.google.com/file/d/1lELLKvdNVMNx0...
4,Nguyen_Hanh_Phuong_Duy_CUSTOMER_SERVICE_ASSOCI...,https://drive.google.com/file/d/1-kxRoNV7pOE7Z...
...,...,...
80,Bui_Thi_Ngoc_Anh_CUSTOMER_SERVICE_ASSOCIATE(04...,https://drive.google.com/file/d/1bn6Pz4sJ339lR...
81,Bui_Eric_Junior_Account_Executive(05-01-2023),https://drive.google.com/file/d/1zzaQdAwrt5p1n...
82,Au_Ngan_CUSTOMER_SERVICE_ASSOCIATE(10-01-2023),https://drive.google.com/file/d/1jcHI-p6xtyVB7...
83,Au_Le_Phoi__San_CUSTOMER_SERVICE_ASSOCIATE(22-...,https://drive.google.com/file/d/1_mufsnlpGoV4Z...


In [164]:
df

,id,date,month,year,company,name,mail,phone_number,position
0,36BB3813,03-02-2023,02,2023,Genkin,Nguyễn Hoàng Ngọc,hoangngoc173@gmail.com,0766476677,CUSTOMER SERVICE ASSOCIATE
1,36BB3813,03-02-2023,02,2023,Genkin,Nguyễn Hoàng Ngọc,hoangngoc173@gmail.com,0766476677,CUSTOMER SERVICE ASSOCIATE
2,36BB3813,03-02-2023,02,2023,Genkin,Nguyễn Hoàng Ngọc,hoangngoc173@gmail.com,0766476677,CUSTOMER SERVICE ASSOCIATE
3,36BB3813,03-02-2023,02,2023,Genkin,Nguyễn Hoàng Ngọc,hoangngoc173@gmail.com,0766476677,CUSTOMER SERVICE ASSOCIATE


In [165]:
df['file_name'] = [file_name_dict[i] for i in df['id']]
data = pd.merge(df,cv_df,how='left')
print(f'{len(data)} new data to append')

4 new data to append


In [166]:
data['source'] = ['CareerBuilder']*len(data)
data['graduated'] = ['']*len(data)
data['note'] = ['']*len(data)
data['name'] = [str(i).upper() for i in data['name']]
data['company'] = [str(i).upper() for i in data['company']]
data['position'] = [str(i).upper() for i in data['position']]

In [167]:
append_index = "A" + str(len(recruitment_data)+2)

if len(data) > 0:
    wks.set_dataframe(data[['id','company','year','month','date','name','source','position','phone_number','mail','cv','graduated','note']], start=append_index, copy_head=False)
print('Done')

Done
